In [ ]:
!pip install -q dart-fss pymupdf

In [ ]:
!pip install pdfplumber

In [47]:
from google.colab import userdata
import os


os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['DART_API_KEY'] = userdata.get('DART_API_KEY')


PINECONE_INDEX_NAME = 'adv-rag'
PINECONE_INDEX_REGION = 'us-east-1'
PINECONE_INDEX_CLOUD = 'aws'
PINECONE_INDEX_METRIC = 'cosine'
PINECONE_INDEX_DIMENSION = 1536

OPENAI_LLM_MODEL = 'gpt-4o-mini'
OPENAI_EMBEDDING_MODEL = 'text-embedding-3-small'

In [ ]:
from google.colab import files

uploaded = files.upload()


In [ ]:
import pdfplumber
import pandas as pd
import os
import re

data = []

# 업로드된 파일 경로 처리
for filename in uploaded.keys():
    # 회사명, 연도 추출
    match = re.match(r'\[(.*?)\]사업보고서\((\d{4})\.\d{2}\.\d{2}\)\.pdf', filename)

    if not match:
        print(f"파일명에서 정보 추출 실패: {filename}")
        continue

    company_name, year = match.groups()
    file_path = filename  # Colab에 업로드된 파일명 그대로 사용

    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"

            if not text.strip():
                print(f"{filename}에서 텍스트 추출 실패 또는 내용 없음")
                continue

            data.append({
                '년도': year,
                '회사명': company_name,
                '텍스트': text.strip()
            })

    except Exception as e:
        print(f"{filename} 처리 중 오류 발생: {e}")

# 데이터프레임 생성 및 CSV 저장
df = pd.DataFrame(data)
output_filename = 'company_reports.csv'
df.to_csv(output_filename, index=False)

print(f"CSV 저장 완료: {output_filename}")
files.download(output_filename)


CSV 저장 완료: company_reports.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
import pandas as pd

# CSV 파일 불러오기
# df = pd.read_csv('company_reports.csv')
df = pd.read_csv('/content/company_reports.csv')

# 각 행의 텍스트 컬럼을 1000자까지만 표시
df['텍스트 미리보기'] = df['텍스트'].str[:100000] + '...'

# 주요 컬럼과 텍스트 미리보기 출력
df_display = df[['년도', '회사명', '텍스트 미리보기']]
df_display.head()


,년도,회사명,텍스트 미리보기
0,2023,신한지주,목 차\n사 업 보 고 서 ..................................
1,2024,신한지주,목 차\n정 정 신 고 (보고)................................
2,2025,신한지주,목 차\n사 업 보 고 서...................................
3,2023,삼성바이오로직스,목 차\n사 업 보 고 서 ..................................
4,2024,삼성바이오로직스,목 차\n사 업 보 고 서...................................


In [11]:
# 한 줄로 전체 텍스트 길이 확인
df['텍스트'].apply(len)

,텍스트
0,1204210
1,1209091
2,1197147
3,383063
4,434157
5,507172
6,368795
7,494292
8,551876
9,553621


In [12]:
# 2023년 삼성전자 사업보고서 앞 1000자 보기
print(df['텍스트'].iloc[0][:1000])


목 차
사 업 보 고 서 .....................................................................................................................1
【 대표이사 등의 확인 】....................................................................................................2
I. 회사의 개요......................................................................................................................3
1. 회사의 개요.....................................................................................................................3
2. 회사의 연혁...................................................................................................................20
3. 자본금 변동사항............................................................................................................35
4. 주식의 총수 등..............................................................................................................36
5. 정관에 관한 사항...........................................................................................................39

In [13]:
# 예: 텍스트 전체 중간쯤(10만자 이후)부터 1000자 출력
start = 100000  # 시작 지점
length = 1000   # 출력할 길이

print(df['텍스트'].iloc[0][start:start+length])


 : 십억원)
구 분 2022년 2021년 2020년
이자수익 1,668.4 1,226.1 804.7
예치금이자 35.9 19.8 18.1
이
유가증권이자 1,207.5 851.8 497.7
자
대출채권이자 420.1 352.0 285.4
부
기타이자수익 4.8 2.6 3.6
문
이자비용(△) 37.0 31.5 31.8
예수부채이자 - - -
전자공시시스템 dart.fss.or.kr Page 67
차입부채이자 0.0 0.0 0.2
사채이자 31.3 28.6 29.7
기타이자비용 5.7 2.8 1.9
소계 1,631.4 1,194.6 772.9
수수료수익 234.0 164.2 71.7
수수료
수수료비용(△) 120.4 53.1 33.5
부문
소계 113.6 111.2 38.2
기타영업수익 7,303.7 5,611.8 4,536.5
보험수익 6,510.4 5,056.9 3,973.7
유가증권평가 및 처분이익 213.1 84.9 110.7
기타 금융자산(부채) 평가 및 처분이익 - - -
배당금수익 246.8 196.2 145.0
외환거래이익 210.0 250.6 134.4
파생상품 관련 이익 108.8 21.9 172.3
대손충당금 환입액 0.4 0.0 -
기
타 충당금 환입액 13.1 0.1 0.4
영 기타 1.2 1.3 0.0
업 기타영업비용 7,936.8 6,211.5 4,861.6
부 보험비용 7,509.1 5,845.4 4,466.8
문
유가증권평가 및 처분손실 50.7 28.8 25.8
기타 금융자산(부채) 평가 및 처분손실 - - -
대손상각비 17.2 13.6 13.1
제충당금 전입액 1.2 8.1 2.0
외환거래손실 102.1 52.9 164.9
파생상품 관련손실 200.2 224.9 157.4
기타 56.3 37.8 31.5
소계 -633.0 -599.7 -325.1
부문별 이익 합계 1,112.0 706.1 486.0
일반관리비(△) 493.9 444.6 255.6
영업이익 618.0 261.4 230.4
영업외이익 -19.1 -16.4 -3.

In [38]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv('company_reports.csv')

# 슬라이싱 함수 정의 (문단 단위 or 토큰 수 기반)
def chunk_text(text, max_chars=1000):
    chunks = []
    while len(text) > max_chars:
        split_idx = text[:max_chars].rfind('\n')  # 최대한 자연스럽게 자르기
        if split_idx == -1:
            split_idx = max_chars
        chunks.append(text[:split_idx].strip())
        text = text[split_idx:].strip()
    if text:
        chunks.append(text)
    return chunks

# 슬라이싱 적용
sliced_data = []

for _, row in df.iterrows():
    chunks = chunk_text(row['텍스트'], max_chars=1000)
    for idx, chunk in enumerate(chunks):
        sliced_data.append({
            '년도': row['년도'],
            '회사명': row['회사명'],
            'chunk_id': idx,
            '내용': chunk
        })

sliced_df = pd.DataFrame(sliced_data)
sliced_df.to_csv('chunked_company_reports.csv', index=False)


In [39]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('chunked_company_reports.csv')

# 텍스트 컬럼 앞 300자 미리보기 생성
df['청크 미리보기'] = df['내용'].str[:300] + '...'

# 주요 컬럼만 보기 좋게 출력
df_display = df[['회사명', '년도', 'chunk_id', '내용']]
df_display.head(10)

,회사명,년도,chunk_id,내용
0,신한지주,2023,0,목 차\n사 업 보 고 서 ..................................
1,신한지주,2023,1,5. 정관에 관한 사항.....................................
2,신한지주,2023,2,1. 요약재무정보........................................
3,신한지주,2023,3,7-2. 증권의 발행을 통해 조달된 자금의 사용실적.....................
4,신한지주,2023,4,3. 주주총회 등에 관한 사항.................................
5,신한지주,2023,5,1. 공시내용 진행 및 변경사항................................
6,신한지주,2023,6,4. 신용파생상품 상세명세 ..................................
7,신한지주,2023,7,본 점 소 재 지 : 서울특별시 중구 세종대로9길 20\n(전 화) 02-6360-...
8,신한지주,2023,8,신용공여 제공이나 후순위사채 매입 등으로 인해 유의적인\n리치게이트엔㈜\n변동이익에...
9,신한지주,2023,9,구조화기업의 성과에 대한 변동이익에 유의적으로 노출되어\n티그리스투자조합50호 있고...


In [ ]:
df[df['회사명'] == '삼성전자'][['년도', 'chunk_id']].value_counts().sort_index()

년도    chunk_id
2023  0           1
      1           1
      2           1
      3           1
      4           1
                 ..
2025  712         1
      713         1
      714         1
      715         1
      716         1
Name: count, Length: 2040, dtype: int64

In [43]:
!pip install -q langchain-openai langchain-pinecone

In [44]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

In [50]:
# 임베딩모델 생성
embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)

# 벡터스토어(클라이언트) 생성
vector_store = PineconeVectorStore(
    index_name=PINECONE_INDEX_NAME,
    embedding=embeddings,
)

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'da4990edcb38e6544f59c43ca2538924', 'date': 'Tue, 15 Jul 2025 07:06:03 GMT', 'server': 'Google Frontend', 'Content-Length': '82', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource adv-rag not found"},"status":404}


In [ ]:
import pandas as pd
from tqdm import tqdm
import tiktoken
import sys

# 토크나이저 불러오기
tokenizer = tiktoken.encoding_for_model("text-embedding-3-small")

# CSV 불러오기
df = pd.read_csv("chunked_company_reports.csv")

documents = []
metadatas = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    documents.append(row["내용"])
    metadatas.append({
        "회사명": row["회사명"],
        "년도": str(row["년도"]),
        "chunk_id": str(row["chunk_id"])
    })

# 제한 조건
MAX_TOKENS = 250000         # OpenAI
MAX_DOCS_PER_BATCH = 100    # Pinecone 용량 대비 보수적으로

batch_docs = []
batch_metas = []
current_tokens = 0

for i in range(len(documents)):
    doc = documents[i]
    meta = metadatas[i]

    token_len = len(tokenizer.encode(doc))

    # 배치 조건 초과 시 업로드
    if (current_tokens + token_len > MAX_TOKENS) or (len(batch_docs) >= MAX_DOCS_PER_BATCH):
        try:
            vector_store.add_texts(texts=batch_docs, metadatas=batch_metas)
            print(f"✅ Uploaded batch: {len(batch_docs)} docs / ~{current_tokens} tokens")
        except Exception as e:
            print(f"❌ Error: {e}")

        # 초기화
        batch_docs = []
        batch_metas = []
        current_tokens = 0

    # 현재 항목 추가
    batch_docs.append(doc)
    batch_metas.append(meta)
    current_tokens += token_len

# 마지막 배치 업로드
if batch_docs:
    try:
        vector_store.add_texts(texts=batch_docs, metadatas=batch_metas)
        print(f"✅ Uploaded final batch: {len(batch_docs)} docs / ~{current_tokens} tokens")
    except Exception as e:
        print(f"❌ Final batch error: {e}")


In [15]:
queries_df = pd.DataFrame({
    "query_id": [f"q{i}" for i in range(1, 11)],
    "query": [
        "2023년 삼성전자 영업이익",
        "2022년 현대차의 부채비율",
        "LG전자의 연간 순이익",
        "네이버의 2023년 매출",
        "카카오의 반기보고서 실적 요약",
        "SK하이닉스의 2023년 2분기 실적은?",
        "2021년 기아의 재무상태 요약",
        "롯데지주의 2023년 자산총계",
        "삼성전자의 매출 추이",
        "LG화학의 순이익 변화"
    ],
    "relevant_doc_ids": [
        ["samsung2023_doc1"],
        ["hyundai2022_doc7"],
        ["lg_doc3"],
        ["naver2023_doc5"],
        ["kakao_half_doc2"],
        ["skhynix_2023q2_doc6"],
        ["kia_2021_doc3"],
        ["lotte_2023_doc9"],
        ["samsung_all_doc_summary"],
        ["lgchem_profit_trend_doc1"]
    ]
})


In [16]:
# 예시 문서 ID (top-5씩)
bm25_candidates = {
    "q1": ["samsung2023_doc1", "samsung2022_doc1", "samsung2021_doc3", "lg_doc3", "naver2023_doc5"],
    "q2": ["hyundai2022_doc7", "hyundai2021_doc2", "kakao_doc1", "naver_doc5", "etc"],
}

dense_candidates = {
    "q1": ["samsung2023_doc1", "naver2023_doc5", "lg_doc3", "random_doc", "etc"],
    "q2": ["hyundai2021_doc2", "hyundai2022_doc7", "etc", "etc", "etc"],
}


In [17]:
def reciprocal_rank_fusion(results_dicts, k=60):
    fused = {}
    for qid in results_dicts[0].keys():
        scores = {}
        for result_dict in results_dicts:
            for rank, doc_id in enumerate(result_dict[qid]):
                scores[doc_id] = scores.get(doc_id, 0) + 1 / (60 + rank)  # RRF 공식
        ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        fused[qid] = [doc_id for doc_id, _ in ranked[:5]]
    return fused

rrf_candidates = reciprocal_rank_fusion([bm25_candidates, dense_candidates])


In [18]:
from sklearn.metrics import precision_score, recall_score

def evaluate_all(results, queries_df):
    metrics = {"P@k": [], "R@k": [], "MRR": [], "MAP": []}

    for _, row in queries_df.iterrows():
        qid = row["query_id"]
        relevant = set(row["relevant_doc_ids"])
        retrieved = results.get(qid, [])

        # P@k
        hit_count = sum([1 for doc in retrieved if doc in relevant])
        metrics["P@k"].append(hit_count / len(retrieved) if retrieved else 0)

        # R@k
        metrics["R@k"].append(hit_count / len(relevant))

        # MRR
        rr = 0
        for rank, doc in enumerate(retrieved):
            if doc in relevant:
                rr = 1 / (rank + 1)
                break
        metrics["MRR"].append(rr)

        # MAP
        ap = 0
        hit = 0
        for rank, doc in enumerate(retrieved):
            if doc in relevant:
                hit += 1
                ap += hit / (rank + 1)
        metrics["MAP"].append(ap / len(relevant) if relevant else 0)

    return {k: round(sum(v) / len(v), 4) for k, v in metrics.items()}


In [19]:
bm25_metrics = evaluate_all(bm25_candidates, queries_df)
dense_metrics = evaluate_all(dense_candidates, queries_df)
rrf_metrics = evaluate_all(rrf_candidates, queries_df)

metrics_df = pd.DataFrame({
    'Metric': ['P@5', 'R@5', 'MRR', 'MAP'],
    'BM25': [bm25_metrics['P@k'], bm25_metrics['R@k'], bm25_metrics['MRR'], bm25_metrics['MAP']],
    'Dense': [dense_metrics['P@k'], dense_metrics['R@k'], dense_metrics['MRR'], dense_metrics['MAP']],
    'RRF': [rrf_metrics['P@k'], rrf_metrics['R@k'], rrf_metrics['MRR'], rrf_metrics['MAP']]
})
metrics_df


,Metric,BM25,Dense,RRF
0,P@5,0.04,0.04,0.04
1,R@5,0.20,0.20,0.20
2,MRR,0.20,0.15,0.15
3,MAP,0.20,0.15,0.15


In [21]:
!pip install langchain-openai
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# 1. OpenAI LLM (gpt-4o-mini)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. ReRank 함수 정의
def rerank_with_llm(query: str, doc_ids: list[str], top_k: int = 5) -> list[str]:
    # 프롬프트 구성 (doc_id를 content처럼 사용)
    prompt = f"질문: {query}\n\n다음 문서들이 주어졌어. 질문과 의미적으로 가장 관련 있는 문서 ID들을 관련도 순으로 정렬해서 최대 {top_k}개만 알려줘:\n\n"
    for i, doc_id in enumerate(doc_ids):
        prompt += f"[{i+1}] 문서 ID: {doc_id}\n"

    prompt += "\n정렬된 문서 ID만 콤마로 나열해서 반환해줘. 예: doc5, doc3, doc1"

    response = llm.invoke(prompt)

    # 결과 파싱
    predicted_ids = [doc.strip() for doc in response.content.split(",") if doc.strip() in doc_ids]
    return predicted_ids[:top_k]


In [23]:
reranked_candidates = {}

for _, row in queries_df.iterrows():
    qid = row["query_id"]
    query = row["query"]

    if qid not in bm25_candidates:
        continue  # 해당 후보 문서 없음 → 넘어감

    doc_ids = bm25_candidates[qid]
    reranked_doc_ids = rerank_with_llm(query, doc_ids, top_k=5)
    reranked_candidates[qid] = reranked_doc_ids


In [24]:
rerank_metrics = evaluate_all(reranked_candidates, queries_df)

metrics_df = pd.DataFrame({
    'Metric': ['P@5', 'R@5', 'MRR', 'MAP'],
    'BM25': [bm25_metrics['P@k'], bm25_metrics['R@k'], bm25_metrics['MRR'], bm25_metrics['MAP']],
    'ReRank(BM25+LLM)': [
        rerank_metrics['P@k'], rerank_metrics['R@k'],
        rerank_metrics['MRR'], rerank_metrics['MAP']
    ]
})
metrics_df


,Metric,BM25,ReRank(BM25+LLM)
0,P@5,0.04,0.04
1,R@5,0.20,0.20
2,MRR,0.20,0.20
3,MAP,0.20,0.20


In [25]:
import pandas as pd
from tqdm import tqdm
import time
from openai import OpenAI
openai = OpenAI()

# 1️⃣ 쿼리셋 (query + 정답 문서 ID)
queries_df = pd.DataFrame({
    "query_id": [f"q{i}" for i in range(1, 11)],
    "query": [
        "2023년 삼성전자 영업이익",
        "2022년 현대차의 부채비율",
        "LG전자의 연간 순이익",
        "네이버의 2023년 매출",
        "카카오의 반기보고서 실적 요약",
        "SK하이닉스의 2023년 2분기 실적은?",
        "2021년 기아의 재무상태 요약",
        "롯데지주의 2023년 자산총계",
        "삼성전자의 매출 추이",
        "LG화학의 순이익 변화"
    ],
    "relevant_doc_ids": [
        ["samsung2023_doc1"],
        ["hyundai2022_doc7"],
        ["lg_doc3"],
        ["naver2023_doc5"],
        ["kakao_half_doc2"],
        ["skhynix_2023q2_doc6"],
        ["kia_2021_doc3"],
        ["lotte_2023_doc9"],
        ["samsung_all_doc_summary"],
        ["lgchem_profit_trend_doc1"]
    ]
})

# 2️⃣ BM25 결과 (top-5)
bm25_candidates = {
    "q1": ["samsung2023_doc1", "samsung2022_doc1", "samsung2021_doc3", "lg_doc3", "naver2023_doc5"],
    "q2": ["hyundai2022_doc7", "hyundai2021_doc2", "kakao_doc1", "naver_doc5", "etc"],
    "q3": ["lg_doc3", "lg_doc1", "samsung2023_doc1", "naver2023_doc5", "etc"],
    "q4": ["naver2023_doc5", "naver2022_doc2", "samsung2023_doc1", "lg_doc3", "etc"],
    "q5": ["kakao_half_doc2", "kakao_doc1", "naver2023_doc5", "lg_doc3", "etc"],
    "q6": ["skhynix_2023q2_doc6", "skhynix_2022_doc1", "samsung2023_doc1", "etc", "etc"],
    "q7": ["kia_2021_doc3", "kia_2020_doc2", "hyundai2021_doc2", "etc", "etc"],
    "q8": ["lotte_2023_doc9", "lotte_2022_doc7", "naver2023_doc5", "etc", "etc"],
    "q9": ["samsung_all_doc_summary", "samsung2023_doc1", "samsung2022_doc1", "etc", "etc"],
    "q10": ["lgchem_profit_trend_doc1", "lg_doc3", "lg_doc1", "etc", "etc"]
}

# 3️⃣ 문서 ID → 본문 텍스트
doc_id_to_text = {
    "samsung2023_doc1": "삼성전자는 2023년 연결기준으로 영업이익 12조 원을 기록했습니다...",
    "samsung2022_doc1": "2022년 삼성전자의 전체 영업이익은 8조 원이며, 반도체 부문이 주요 기여를 했습니다...",
    "samsung2021_doc3": "2021년에는 코로나19에도 불구하고 안정적인 실적을 유지했습니다...",
    "samsung_all_doc_summary": "삼성전자의 2020~2023년 매출 추이는 전반적으로 상승세를 보였으며, 2023년 정점을 기록했습니다...",
    "lg_doc3": "LG전자의 연간 순이익은 3조 원으로, 전년도 대비 15% 증가했습니다...",
    "lg_doc1": "LG전자는 생활가전 부문에서 안정적인 매출을 올렸습니다...",
    "naver2023_doc5": "네이버는 2023년 매출 9조 원을 기록하며 역대 최고 실적을 달성했습니다...",
    "naver2022_doc2": "2022년 네이버의 주요 성장 동력은 커머스와 광고 부문이었습니다...",
    "kakao_half_doc2": "2023년 상반기 기준 카카오의 실적은 전년 대비 10% 상승하였습니다...",
    "kakao_doc1": "카카오의 주력 사업은 메신저와 광고이며, 수익이 증가하고 있습니다...",
    "skhynix_2023q2_doc6": "SK하이닉스는 2023년 2분기에 영업이익 1조 5천억 원을 기록했습니다...",
    "skhynix_2022_doc1": "2022년 SK하이닉스는 메모리 반도체 수요 감소로 실적이 하락했습니다...",
    "hyundai2022_doc7": "2022년 현대자동차의 부채비율은 128%로, 전년도 대비 안정적인 수준을 유지했습니다...",
    "hyundai2021_doc2": "2021년 현대차는 전기차 중심의 전략으로 해외 시장을 공략했습니다...",
    "kia_2021_doc3": "기아는 2021년 총자산 대비 부채비율이 142%였으며, 수익성 지표가 개선되었습니다...",
    "kia_2020_doc2": "2020년 기아는 전년 대비 순이익이 감소했지만, 구조조정을 통해 회복 기조를 보였습니다...",
    "lotte_2023_doc9": "2023년 롯데지주의 자산총계는 약 18조 원 수준이었습니다...",
    "lotte_2022_doc7": "2022년 대비 자산은 5% 증가했고, 투자 활동이 활발했습니다...",
    "lgchem_profit_trend_doc1": "LG화학의 순이익은 2021년부터 2023년까지 꾸준히 증가했습니다. 2023년 순이익은 2.8조 원입니다...",
}

# 4️⃣ ReRank (문서 내용 기반)
def rerank_with_llm_contents(query, doc_ids, doc_id_to_text, top_k=5):
    prompt = f"질문: '{query}'\n\n다음은 관련 문서들입니다. 질문에 가장 관련 있는 순서대로 1~5위를 매겨주세요:\n\n"
    for idx, doc_id in enumerate(doc_ids):
        content = doc_id_to_text.get(doc_id, "내용 없음")
        prompt += f"[{idx+1}] 문서 ID: {doc_id}\n본문: {content[:800]}\n\n"
    prompt += "출력 형식: 가장 관련 있는 문서 ID 5개를 순서대로 콤마로 나열하세요."

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    content = response.choices[0].message.content.strip()
    return [doc.strip() for doc in content.split(",") if doc.strip() in doc_ids][:top_k]

# 5️⃣ 전체 ReRank 수행
reranked_results = {}
for _, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
    qid = row["query_id"]
    query = row["query"]
    doc_ids = bm25_candidates.get(qid, [])
    if not doc_ids:
        continue
    reranked_results[qid] = rerank_with_llm_contents(query, doc_ids, doc_id_to_text)
    time.sleep(1.2)  # API 호출 간격

# 6️⃣ 평가 함수
def evaluate_all(results, queries_df):
    metrics = {"P@k": [], "R@k": [], "MRR": [], "MAP": []}
    for _, row in queries_df.iterrows():
        qid = row["query_id"]
        relevant = set(row["relevant_doc_ids"])
        retrieved = results.get(qid, [])

        # P@k
        hit_count = sum([1 for doc in retrieved if doc in relevant])
        metrics["P@k"].append(hit_count / len(retrieved) if retrieved else 0)

        # R@k
        metrics["R@k"].append(hit_count / len(relevant))

        # MRR
        rr = 0
        for rank, doc in enumerate(retrieved):
            if doc in relevant:
                rr = 1 / (rank + 1)
                break
        metrics["MRR"].append(rr)

        # MAP
        ap = 0
        hit = 0
        for rank, doc in enumerate(retrieved):
            if doc in relevant:
                hit += 1
                ap += hit / (rank + 1)
        metrics["MAP"].append(ap / len(relevant) if relevant else 0)
    return {k: round(sum(v) / len(v), 4) for k, v in metrics.items()}

# 7️⃣ 성능 비교
bm25_metrics = evaluate_all(bm25_candidates, queries_df)
rerank_metrics = evaluate_all(reranked_results, queries_df)

metrics_df = pd.DataFrame({
    'Metric': ['P@5', 'R@5', 'MRR', 'MAP'],
    'BM25': [bm25_metrics['P@k'], bm25_metrics['R@k'], bm25_metrics['MRR'], bm25_metrics['MAP']],
    'ReRank(BM25+LLM)': [rerank_metrics['P@k'], rerank_metrics['R@k'], rerank_metrics['MRR'], rerank_metrics['MAP']]
})
metrics_df


100%|██████████| 10/10 [00:18<00:00,  1.87s/it]


,Metric,BM25,ReRank(BM25+LLM)
0,P@5,0.2,0.2
1,R@5,1.0,1.0
2,MRR,1.0,1.0
3,MAP,1.0,1.0


In [26]:
import pandas as pd
from tqdm import tqdm
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# 1. 쿼리셋 정의
queries_df = pd.DataFrame({
    "query_id": [f"q{i}" for i in range(1, 6)],
    "query": [
        "2023년 삼성전자 영업이익",
        "2022년 현대차의 부채비율",
        "LG전자의 연간 순이익",
        "네이버의 2023년 매출",
        "카카오의 반기보고서 실적 요약",
    ],
    "relevant_doc_ids": [
        ["samsung2023_doc1", "samsung2023_doc2"],
        ["hyundai2022_doc7", "hyundai2022_doc8"],
        ["lg_doc3", "lg2023_doc1"],
        ["naver2023_doc5", "naver2023_summary_doc2"],
        ["kakao_half_doc2", "kakao_2023_doc1"],
    ]
})

# 2. BM25 후보 문서 ID
bm25_candidates = {
    "q1": ["samsung2023_doc1", "samsung2022_doc1", "samsung2023_doc2", "lg_doc3", "naver2023_doc5"],
    "q2": ["hyundai2022_doc7", "hyundai2021_doc2", "hyundai2022_doc8", "naver_doc5", "etc"],
    "q3": ["lg2023_doc1", "lg_doc3", "random_doc", "etc", "etc"],
    "q4": ["naver2023_doc5", "naver2023_summary_doc2", "naver2022_doc4", "kakao_doc1", "etc"],
    "q5": ["kakao_half_doc2", "kakao_2023_doc1", "kakao_2022_doc1", "naver_doc5", "etc"],
}

# 3. 문서 내용 정의
documents = {
    "samsung2023_doc1": "삼성전자의 2023년 영업이익은 20조 원이었습니다.",
    "samsung2023_doc2": "2023년 삼성전자 실적 발표에 따르면 영업이익이 큰 폭으로 증가했습니다.",
    "hyundai2022_doc7": "현대자동차의 2022년 부채비율은 130%였습니다.",
    "hyundai2022_doc8": "2022년 현대차의 부채비율이 증가했으며 이는 투자 확대에 기인합니다.",
    "lg_doc3": "LG전자의 2021년 순이익은 2조 원입니다.",
    "lg2023_doc1": "2023년 LG전자의 연간 순이익은 3조 원이었습니다.",
    "naver2023_doc5": "네이버의 2023년 연간 매출은 9조 원을 돌파했습니다.",
    "naver2023_summary_doc2": "2023년 네이버 실적 요약: 매출 상승 및 사용자 증가.",
    "kakao_half_doc2": "2023년 상반기 카카오의 실적은 전년 대비 소폭 증가했습니다.",
    "kakao_2023_doc1": "카카오 반기보고서에 따르면 실적은 안정적인 성장세를 보였습니다.",
    "samsung2022_doc1": "삼성전자 2022년 실적 요약입니다.",
    "hyundai2021_doc2": "2021년 현대차의 부채비율은 120%였습니다.",
    "naver_doc5": "과거 네이버 사업 보고서 내용입니다.",
    "kakao_2022_doc1": "2022년 카카오의 실적은 견조한 흐름을 보였습니다.",
    "kakao_doc1": "카카오의 일반 정보입니다.",
    "etc": "기타 문서 내용입니다.",
    "random_doc": "무관한 내용의 문서입니다."
}

# 4. ReRank 함수 정의
llm = ChatOpenAI(model="gpt-4o", temperature=0)

def rerank_with_llm(query, doc_ids, top_k=5):
    doc_texts = [f"[{doc_id}]: {documents.get(doc_id, '')}" for doc_id in doc_ids]
    context = "\n\n".join(doc_texts)
    prompt = f"""
질문: "{query}"
다음은 관련된 문서 목록입니다. 각 문서는 ID와 내용으로 구성되어 있습니다. 질문과 가장 관련 있는 문서를 상위 {top_k}개 골라 ID 순서대로 나열하세요. ID만 콤마로 나열해주세요.

{context}
"""
    response = llm([HumanMessage(content=prompt)]).content.strip()
    return [doc_id.strip() for doc_id in response.split(",") if doc_id.strip() in doc_ids]

# 5. 모든 쿼리에 대해 ReRank 수행
reranked_results = {}
for _, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
    qid = row["query_id"]
    query = row["query"]
    doc_ids = bm25_candidates[qid]
    reranked_doc_ids = rerank_with_llm(query, doc_ids)
    reranked_results[qid] = reranked_doc_ids

# 6. 평가 함수 정의
def evaluate_all(results, queries_df):
    metrics = {"P@k": [], "R@k": [], "MRR": [], "MAP": []}
    for _, row in queries_df.iterrows():
        qid = row["query_id"]
        relevant = set(row["relevant_doc_ids"])
        retrieved = results.get(qid, [])
        hit_count = sum([1 for doc in retrieved if doc in relevant])
        metrics["P@k"].append(hit_count / len(retrieved) if retrieved else 0)
        metrics["R@k"].append(hit_count / len(relevant))
        rr = 0
        for rank, doc in enumerate(retrieved):
            if doc in relevant:
                rr = 1 / (rank + 1)
                break
        metrics["MRR"].append(rr)
        ap = 0
        hit = 0
        for rank, doc in enumerate(retrieved):
            if doc in relevant:
                hit += 1
                ap += hit / (rank + 1)
        metrics["MAP"].append(ap / len(relevant) if relevant else 0)
    return {k: round(sum(v) / len(v), 4) for k, v in metrics.items()}

# 7. BM25 원본 및 ReRank 성능 비교
bm25_metrics = evaluate_all(bm25_candidates, queries_df)
rerank_metrics = evaluate_all(reranked_results, queries_df)

# 8. 지표 테이블 출력
metrics_df = pd.DataFrame({
    'Metric': ['P@5', 'R@5', 'MRR', 'MAP'],
    'BM25': [bm25_metrics['P@k'], bm25_metrics['R@k'], bm25_metrics['MRR'], bm25_metrics['MAP']],
    'ReRank(BM25+LLM)': [rerank_metrics['P@k'], rerank_metrics['R@k'], rerank_metrics['MRR'], rerank_metrics['MAP']]
})
metrics_df


/tmp/ipython-input-26-4272580194.py:56: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", temperature=0)
  0%|          | 0/5 [00:00<?, ?it/s]/tmp/ipython-input-26-4272580194.py:67: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=prompt)]).content.strip()
100%|██████████| 5/5 [00:05<00:00,  1.07s/it]


,Metric,BM25,ReRank(BM25+LLM)
0,P@5,0.4000,0.9333
1,R@5,1.0000,1.0000
2,MRR,1.0000,1.0000
3,MAP,0.9333,1.0000


In [27]:
!pip uninstall -y pinecone-client
!pip install -U pinecone langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 11.5 MB/s eta 0:00:00


In [28]:
!pip uninstall -y pinecone-client pinecone

Found existing installation: pinecone 7.3.0
Uninstalling pinecone-7.3.0:
  Successfully uninstalled pinecone-7.3.0


In [29]:
!pip install pinecone langchain-openai

  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
Using cached pinecone-7.3.0-py3-none-any.whl (587 kB)


In [30]:
from pinecone import Pinecone, ServerlessSpec

print("✅ 최신 pinecone 설치 확인 완료")

✅ 최신 pinecone 설치 확인 완료


In [32]:
!pip uninstall pinecone-client -y
!pip uninstall pinecone -y


Found existing installation: pinecone 7.3.0
Uninstalling pinecone-7.3.0:
  Successfully uninstalled pinecone-7.3.0


In [33]:
!pip install -U pinecone-client langchain-openai langchain langchain-pinecone

  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 1.9 MB/s eta 0:00:00
Using cached pinecone-7.3.0-py3-none-any.whl (587 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 8.7 MB/s eta 0:00:00


In [34]:
pip install pinecone

In [35]:
# ✅ 환경 변수 설정
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")

# ✅ 임포트
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA

# ✅ Pinecone 객체 생성 및 인덱스 불러오기
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index("adv-rag")  # 반드시 존재하는 index 이름

# ✅ 임베딩 모델
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# ✅ LangChain VectorStore로 wrapping
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embedding_model,
    text_key="text"
)

# ✅ 리트리버 설정
retriever = vectorstore.as_retriever(search_type="similarity", k=5)

# ✅ LLM 연결
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# ✅ QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# ✅ 테스트용 질의
query = "삼성전자의 최근 분기 실적이 어땠어?"
result = qa_chain.invoke({"query": query})

# ✅ 출력
print("📌 질문:", query)
print("\n🧠 답변:\n", result["result"])
print("\n📂 근거 문서 개수:", len(result["source_documents"]))


NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '9473a7ea1493706d5edea410ef00a6cb', 'date': 'Tue, 15 Jul 2025 06:50:08 GMT', 'server': 'Google Frontend', 'Content-Length': '82', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource adv-rag not found"},"status":404}


In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

import os

# 🔑 환경 변수
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
os.environ['PINECONE_API_KEY'] = userdata.get("PINECONE_API_KEY")

# ✅ 임베딩 모델 설정
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# ✅ Pinecone 연결
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
index = pc.Index("adv-rag")
vectorstore = PineconeVectorStore(index, embedding=embedding_model, text_key="text")


In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# ✅ Prompt 템플릿 - 🔥context를 반드시 반영하도록 지시
template = """
당신은 재무 보고서를 바탕으로 답변하는 전문 AI입니다.

다음 문서를 참고하여 사용자 질문에 정확하고 구체적으로 답변하세요.
반드시 아래 context의 내용을 기반으로만 답변하고, 모르면 모른다고 말하세요.

<문서>
{context}
</문서>

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)


NameError: name 'vectorstore' is not defined

In [ ]:
query = "네이버의 주요 사업 부문은 어떻게 구성되어 있어?"
response = rag_chain.invoke(query)
print(response.content)

네이버의 주요 사업 부문은 다음과 같이 구성되어 있습니다:

1. **서치플랫폼**: 검색과 디스플레이 광고로 구성되어 있으며, 이용자의 정보탐색 수요를 연결시켜 비즈니스 활동을 촉진하는 사업입니다.

2. **커머스**: 커머스 광고, 중개 및 판매, 멤버십 서비스를 포함합니다.

3. **핀테크**: 페이 서비스 및 디지털 금융 플랫폼을 제공합니다.

4. **콘텐츠**: 웹툰, 제페토, SNOW 등의 콘텐츠 서비스를 포함합니다.

5. **클라우드**: NCP, 웍스, 클로바 등 다양한 IT 인프라 및 솔루션을 제공합니다.

이러한 사업 부문들은 네이버의 다각화된 사업 포트폴리오를 형성하고 있으며, 안정적인 성장을 이어가고 있습니다.


In [ ]:
query = "네이버 글로벌 사업 전략"
response = rag_chain.invoke(query)
print(response.content)

네이버의 글로벌 사업 전략은 다음과 같습니다:

1. **글로벌 웹툰 플랫폼 운영**: 네이버는 네이버웹툰(한국), LINE Manga(일본), WEBTOON(미국/유럽), LINE WEBTOON(동남아) 등 다양한 웹툰 플랫폼을 운영하여 글로벌 시장에서의 입지를 강화하고 있습니다. 이를 통해 100여 개국에서 다운로드 및 수익 랭킹에서 수위권을 유지하고 있습니다.

2. **콘텐츠 사업 확장**: 네이버는 방대한 원작 IP를 활용하여 영화 및 드라마 제작을 위한 스튜디오N을 설립하고, 영상화 사업을 강화하고 있습니다. 또한, VFX 및 애니메이션 제작 역량을 갖춘 로커스를 인수하여 경쟁력을 높이고 있습니다.

3. **시장 다각화**: 네이버는 일본과 미국 등 서구권 시장을 공략하기 위해 LINE Digital Frontier를 인수하고, 글로벌 현지 작가와 작품 발굴을 위해 투자를 확대하고 있습니다. 이는 해외 웹툰 시장의 성장 가능성을 염두에 둔 전략입니다.

4. **기술 및 서비스 혁신**: 인공지능, 로보틱스 등 미래 기술에 대한 지속적인 연구개발을 통해 기술 플랫폼의 변화와 혁신을 추구하고 있습니다. 이를 통해 글로벌 시장에서의 경쟁력을 강화하고 있습니다.

5. **비용 효율화 및 수익성 개선**: 2023년에는 비용 효율화에 초점을 맞추어 전 사업부문의 내실을 다지며, 영업수익과 영업이익을 증가시키는 성과를 거두었습니다. 이러한 전략은 지속 가능한 성장을 위한 기반을 마련하는 데 기여하고 있습니다.

이러한 전략을 통해 네이버는 글로벌 시장에서의 경쟁력을 지속적으로 강화하고 있습니다.


In [ ]:
query = "삼성전자 반도체 부문 실적"
response = rag_chain.invoke(query)
print(response.content)


삼성전자의 2023년 반도체 부문 누적 생산실적은 34,806,295백만원으로 집계되었습니다. 이 기간 동안 생산설비의 평균 가동률은 100%를 유지하였습니다. 2022년과 2021년의 반도체 부문 생산실적은 각각 33,326,132백만원과 25,626,444백만원이었습니다.


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough

template = """
당신은 재무 보고서를 바탕으로 답변하는 전문 AI입니다.

다음 문서를 참고하여 사용자 질문에 정확하고 구체적으로 답변하세요.
반드시 아래 context의 내용을 기반으로만 답변하고, 모르면 모른다고 말하세요.

<문서>
{context}
</문서>

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

non_rag_chain = (
    {"context": lambda x: static_context, "question": RunnablePassthrough()}
    | prompt
    | llm
)

# 사용 예시
query = "네이버의 주요 사업 부문은 어떻게 구성되어 있어?"
response = non_rag_chain.invoke(query)
print(response.content)

/tmp/ipython-input-13-3682590899.py:24: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


죄송하지만, 제공된 문서에는 재무 정보나 네이버의 주요 사업 부문에 대한 내용이 포함되어 있지 않습니다. 따라서 해당 질문에 대한 답변을 드릴 수 없습니다.
